In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import thinkstats2
import thinkplot
from sklearn.metrics import mean_squared_error
import scipy.stats as ss

import statsmodels.formula.api as sm

<H1>Central Limit Theorem and Non-Normal Data</h1>

The Central Limit Theorem is one of the important concepts of statistics, and something that we have address in a backdoor way while doing simulations. The CLT states:
<ul>
<li>When independent random samples are taken, the distribution of their means approaches normal as more samples are taken. 
<li>The mean of the sample will equal the mean of the population. 
<li>The standard deviation of the sample will equal the standard deviation of the population divided by the root of the sample size.
</ul>

We can use this as a shortcut to help analyze data that is not normal, or where we don't know the distribution. 

In [ ]:
#Note: the filter here takes out large values. This works fine either way, but with the large outliers in there, visualizing
#is less effective, due to the range. 
# Also, if a sample includes one $20 property randomly, that will have a big impact. 
df = pd.read_csv("Assessments.csv")
df = df[df['Assessed Value']<1000000]
df = df[df['Assessed Value']>20000]
df.sort_values("Assessed Value", ascending=False).head()

In [ ]:
#Visualize Distribution and a normality plot
thinkplot.PrePlot(2,1,2)
sns.histplot(df["Assessed Value"])
thinkplot.SubPlot(2)
thinkstats2.NormalProbabilityPlot(df["Assessed Value"])
thinkplot.Config()

Based on both visualizations, the data is pretty clearly not normal. 

How do we deal with this non-normal data? We have the data of the population here, so we can just calculate it. In reality though, this isn't normally the case. Think about looking at items where we can't capture the population data and are relying on a sample exclusively:
<ul>
<li>The gorilla example from the book, where we can only tranquilize and assess a small fraction of the total populations.
<li>Any survey type data, short of a full census. We only get responses from part of the population.
<li>Scientific measurements, where we have only a few examples.
</ul>

In any of these cases we don't have the distribution of the population, and we really can't get it in any reasonable way. This is where the CLT comes in handy - we can trust the the results of our samples will be normal, then use that to infer about the entire population. The distribution of the means being normal means that we can rely on all the implicit powers of the normal distribution analytical tools.

We can call back to our estimation days and generate some samples...

In [ ]:
#Take some samples 
i = 0
n = 100
total = 1000
means = []

while i < total:
    tmp = df["Assessed Value"].sample(n)
    means.append(tmp.mean())
    i += 1

thinkplot.PrePlot(2,1,2)
sns.histplot(means, binwidth=5000, kde=True)
thinkplot.SubPlot(2)
thinkstats2.NormalProbabilityPlot(means)
thinkplot.Config()

<h3>Sampling Results</h3>

Taking a bunch of samples delivered us a stack of means. We can run some calculations to see how well we did. The means should be pretty close, that's part of the power of the CLT! The standard deviation of the samples are also the standard error, indicating our expected accuracy. If we increase the n, that SE should go down.

In [ ]:
popMean = df["Assessed Value"].mean()
popStd = df["Assessed Value"].std()
estMean = np.mean(means)
estSE = np.std(means)
print("Population Mean:", popMean)
print("Population Std:", popStd)
print("\n")
print("Estimate Mean:", estMean)
print("Estimate SE:", estSE)
print("Estimate Std:", estSE*np.sqrt(n))

<h3>Normality status - Increased!</h3>

The means of the samples are pretty normal, even if the underlying distribution isn't. The normal distribution is common (hence the normal part), but everything in life isn't normally distributed, as we've seen. 
<ul>
<li>Incomes can often be represented with a lognormal distribution. 
<li>Property assessments here have a bimodal distribution (two humps).
<li>Some things we may want to look at have a distribution that we just don't know at all!
</ul>


<h2>CLT in Full Effect</h2>

The example above shows the idea behind the CLT - we can take samples and use them to estimate for a population. We are comfortable with this idea from all the estimation stuff. 

The analytical piece of the CLT is also really handy - we can replicate similar results without all the sampling. Taking 100 or 1000 samples presents similar problems to trying to measure the whole population - it can be hard, time consuming, expensive, or impossible. We can build a function that calculates the CLT directly, and compare the results. 

In [ ]:
#Calculable CLT
def clt(data):
    mean = np.mean(data)
    se = np.std(data) / np.sqrt(len(data))
    return (mean, se)

In [ ]:
#Run CLT Calculation with 1 Sample
#Compare with results above
cltRes = clt(df["Assessed Value"].sample(n))
print("CLT Mean:", cltRes[0])
print("CLT SE:", cltRes[1])
print("CLT Est Std:", cltRes[1]*np.sqrt(n))
print("\n")
print("Population Mean:", popMean)
print("Population Std:", popStd)
print("\n")
print("Estimate Mean:", estMean)
print("Estimate SE:", estSE)
print("Estimate Std:", estSE*np.sqrt(n))

In [ ]:
#Exercise

#Rerun with a few different N values to see the changes 

<h3>CLT is Quick</h3>

We can run a bunch of simulations like we did while estimating, but this is way quicker!

Try it for some random distribution...

In [ ]:
#Generate random Exponential Dist - put into a Series for ease of use later. 
expoDist = pd.Series(ss.expon.rvs(size=10000))
sns.histplot(expoDist)

In [ ]:
cltExp = clt(expoDist.sample(n))
print("CLT Mean:", cltExp[0])
print("CLT SE:", cltExp[1])
print("CLT Est Std:", cltExp[1]*np.sqrt(n))
print("\n")
print("Dist Mean:", expoDist.mean())
print("Dist Std:", expoDist.std())

That's the magic of the CLT - we can rely on our smallish sample far more than we might think! 

Because this works, we can also harness some of the other useful assumptions of normality, like confidence intervals. Going back to the assessments from above, how confident can we be about our estimate of the mean? What is the 90 confidence interval?

<h3>Back to the Assessments</h3>

In [ ]:
#Directly calculate confidence intervals.
ci1low = ss.norm.ppf(.05, loc=cltRes[0], scale=cltRes[1])
ci1hi = ss.norm.ppf(.95, loc=cltRes[0], scale=cltRes[1])
ci1low, ci1hi

And if we compare that to the predictions based on estimation?

In [ ]:
cdf = thinkstats2.Cdf(means)
cdf.Percentile(.05), cdf.Percentile(.95)

The exact difference will vary a bit, due to randomness, but we should expect the CI bands to be slightly worse than the estimation example - but we only did one sample! Using the CLT allows us to much more quickly infer about the population when we don't have access to large numbers of samples. 

<h3>Try one</h3>

Use the CLT to calculate the mean and std from a sample of the salary column below. 

In [ ]:
df_sal = pd.read_csv("spain_salary.csv")
df_sal.head()

Plot the data, is it normal? 

In [ ]:
#Plot Distribution

use CLT to generate mean and std estimates. Compare to a direct calcuation of the population. 

In [ ]:
#Use CLT to infer mean and std

And the 85% confidence interval?

In [ ]:
#Generate 85% CI for salary
#85% = 7.5 to 92.5


<h1>Common Transformations - Box Cox</h1>

Another strategy to deal with non-normal data is to just do a transformation to make it more normal. We saw this before in one of the sample datasets where we were provided with a log_income value, which changed a lognormal distribution of income into a normal distribution that is easier to analyze. 

These types of transformations are somewhat of a one-off, what makes a distribution normal will differ. One common method is Box-Cox, which raises values to some value - lambda. E.g. if lambda is 2, the original data is squared, if lambda is .5 the original data is square rooted. Lambda can be between -5 and 5. 

This process is something that we can utilize scipy to automate - it will give us back a transformation and the optimal lambda. It tests multiple values of lambda, and determines which is most normal using a normality test. 

In [ ]:
#Load some data
def process_time(row):
    
    call_received = datetime.datetime.strptime(row['date_time_received'].split('+')[0], '%Y-%m-%dT%H:%M:%S')
    call_ended    = datetime.datetime.strptime(row['date_time_complete'].split('+')[0], '%Y-%m-%dT%H:%M:%S')
    
    time_ellapsed = call_ended - call_received

    row['Parsed Call Received'] = str(call_received)
    row['Parsed Call Ended'] = str(call_ended)
    row['Time Ellapsed'] = str(time_ellapsed)
    row['Time Ellapsed (minutes)'] = round(time_ellapsed.total_seconds() / 60, 1)

    return row

import requests
import io
import datetime
base_url = 'https://aegis4048.github.io/downloads/notebooks/sample_data/'
filename = '08c32c03-9d88-42a9-b8a1-f493a644b919_NRCEventReporting-Calls-2010.csv'
data = requests.get(base_url + filename).content
df_   = pd.read_csv(io.StringIO(data.decode('utf-8')))

parsed_df = df_.iloc[11000: 12000, :].apply(process_time, axis=1).iloc[:, -4:]
parsed_df['Parsed Call Received'] = pd.to_datetime(parsed_df['Parsed Call Received'], format='%Y-%m-%d %H:%M:%S')
parsed_df['Parsed Call Ended'] = pd.to_datetime(parsed_df['Parsed Call Ended'], format='%Y-%m-%d %H:%M:%S')
parsed_df = parsed_df.sort_values(by = 'Parsed Call Received')

new_parsed_df = parsed_df.drop_duplicates(subset=['Parsed Call Received'], keep=False)
df2 = new_parsed_df.copy()
df2.head()

In [ ]:
#Visualize call time
calls = pd.Series(df2["Time Ellapsed (minutes)"])
sns.histplot(calls, kde=True)

In [ ]:
transCalls, lmbda = ss.boxcox(calls)
print(lmbda)
sns.histplot(transCalls, kde=True)

Much more normal!

<h3>Application - Control Charts</h3>

One use of a transformation like this would be to monitor and detect things that fall outside the normal expected bounds. This data is from a call center, we may want to see when calls fall outside a specified range. Going in however, we don't have a cutoff - we can use the distribution to tell us how many are outside of 3std. This is common in lots of quality control/manufacturing types of scenarios. 

The control chart has a normality assumption, because the idea behind it is based off of the standard deviation. We want to "catch" things that are over X standard deviations from normal. If you've heard of Six Sigma, that's based on the same idea - to manufacture things to "six sigmas" of quality (or without error): roughly 99.99966%. This type of transformation to normal for a control chart is a common thing. For more info: https://www.isixsigma.com/tools-templates/control-charts/non-normal-data-needs-alternate-control-chart-approach/ 

In [ ]:
#Set # of standard deviations to draw control lines. 
limit = 2.5

In [ ]:
#Non-transformed control chart. 
y = new_parsed_df['Parsed Call Received'].values
mean = np.mean(calls)
std  = np.std(calls)

upper_limit = mean + limit * std
lower_limit = mean - limit * std
fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(y, calls)

ax.axhline(mean, color='C1')
ax.axhline(upper_limit, color='r')
ax.axhline(lower_limit, color='r');
ax.text(y[-200], upper_limit + 3, 'Upper Control Limit', color='r')
ax.text(y[-200], lower_limit + 3, 'Lower Control Limit', color='r')
ax.text(y[3], mean + 3, 'Mean', color='C1')
ax.set_ylabel('Call duration (minutes)');
ax.set_title('Control Chart for Phone Call Duration - Original');

This isn't super helpful. The lower limits are negative, the control limits seem random. Not great. 

We can make one with transformed data. 

In [ ]:
#Transformed control chart. 
mean_trans = np.mean(transCalls)
std_trans  = np.std(transCalls)

upper_limit_trans = mean_trans + limit * std_trans
lower_limit_trans = mean_trans - limit * std_trans
fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(y, transCalls)

ax.axhline(mean_trans, color='C1')
ax.axhline(upper_limit_trans, color='r')
ax.axhline(lower_limit_trans, color='r');
ax.text(y[-200], upper_limit_trans - 0.15, 'Upper Control Limit', color='r')
ax.text(y[-200], lower_limit_trans + 0.15, 'Lower Control Limit', color='r')
ax.text(y[3], mean_trans + 0.1, 'Mean', color='C1')
ax.set_ylabel('Call duration (minutes)');
ax.set_title('Control Chart for Phone Call Duration - Transformed');

Way better! Now the control lines are useful. This is something that is really done in practice. 

Now, we don't really care about the transformed data specifically, we care about the real values. How do we go backwards? Call the inverse transformation. 

In [ ]:
#Reverse the transformation. 
from scipy.special import inv_boxcox
backTrans = inv_boxcox(transCalls, lmbda)
sns.histplot(backTrans, kde=True)

Voila! Back where we started. 

Why? We can transform some data to make it normal to utilize any normal-based analysis on the data. We can more easily do hypothesis testing, control charts, confidence intervals, etc...

Using the benefits of normality can help us out!

Another similar example is Yeo-Johnson: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.yeojohnson.html 

There's also QunitileTransformer, but it requires far more data: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html#sklearn.preprocessing.QuantileTransformer 